In [1]:
import torch
import random
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [2]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [4]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  
  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [10]:
# boilerplate done, relevant code:

In [11]:
# utility function we will use later when comparing manual gradients to PyTorch gradients
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [42]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 64 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1 # using b1 just for fun, it's useless because of BN
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

# Note: I am initializating many of these parameters in non-standard ways
# because sometimes initializating with e.g. all zeros could mask an incorrect
# implementation of the backward pass.

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

4137


In [43]:
batch_size = 32
n = batch_size # a shorter variable also, for convenience
# construct a minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

In [ ]:
# forward pass, "chunkated" into smaller steps that are possible to backward one at a time

emb = C[Xb] # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation
# BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-linearity
h = torch.tanh(hpreact) # hidden layer
# Linear layer 2
logits = h @ W2 + b2 # output layer
# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# PyTorch backward pass
for p in parameters:
  p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss

tensor(3.3463, grad_fn=<NegBackward0>)

In [54]:
-logprobs[range(n), Yb]

tensor([3.9584, 3.2039, 3.6897, 3.1886, 4.1585, 3.3932, 3.1447, 4.1854, 3.1880,
        4.2471, 3.1214, 1.6884, 2.7603, 3.1157, 2.9421, 3.2532, 3.7826, 2.9890,
        3.6095, 3.3761, 2.8744, 2.9738, 4.2653, 3.9974, 3.5157, 2.8570, 3.0641,
        3.9162, 2.7416, 3.4218, 3.3582, 3.0993], grad_fn=<NegBackward0>)

In [55]:
-logprobs[range(n), Yb].mean()

tensor(3.3463, grad_fn=<NegBackward0>)

In [62]:
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1/n
dlogprobs

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         -0.0312,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.0312,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.0312,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000

In [153]:
# Exercise 1: backprop through the whole thing manually, 
# backpropagating through exactly all of the variables 
# as they are defined in the forward pass above, one by one

# -----------------
# YOUR CODE HERE :)
# -----------------

# loss = -logprobs[range(n), Yb].mean()
# deriv of -mean is -1/n (see proof written out)
# This is a bit confusing because ultimately, we're looking at
# "negative the mean of logprobs, indexed"
# To address the indexed part, we just ignore all the non-index values
# aka set them to zero, and set the relevant values (that we took the mean of)
# all to -1/n, so where n=32 as here, -1/32 = -0.0312

# The other confusing thing is the nomenclature
# What we're getting here is the derivative of loss with respect to (w.r.t) logpropbs
# so this is dloss/dlogprobs, which is referenced as "dlogprobs" evidently
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1/n
cmp('logprobs', dlogprobs, logprobs)

# logprobs = probs.log()
# deriv of log(probs) (in pytorch, log is natural log aka log base e)
# is 1/probs
# This gives us the local derivative of logprobs. We can now apply the chain rule to get the global derivative by multiplying dlogprobs
dprobs = 1 / probs * dlogprobs
cmp('probs', dprobs, probs)

# probs = counts * counts_sum_inv
# This one is tricky due to pytorch broadcasting
# counts * dprobs is correct if we were just doing scalar values
# But in pytorch, we have 
# counts.shape = [32, 27] and counts_sum_inv.shape = [32, 1]
# So a[3,3] * b[3,1] -->
# a11*b1 a12*b1 a13*b1 etc
# b1 is being used multiple times, so we need to *add* the derivative
# in each case it is used. So we take the deriv of each use
#  (counts * dprobs), and we add it up, accumulating the gradient.
# We also make sure to keep the dimensions intact (sum would otherwise
# drop the last dimention)
dcounts_sum_inv = (counts * dprobs).sum(1, keepdim=True)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)

# probs = counts * counts_sum_inv
# This is tricky again. 
# We have to do the derivative with respect to
# each term in this equation. We just did dcounts_sum_inv, so now we still have
# to do dcounts. 
# This is multiplication again, so the derivative is just the other term.
# **However**, the way the gradient accumulates is different.
# We must inspect the shapes of the values in the original equation
# (probs = counts * counts_sum_inv) to determine how we accumulate:
# So again:
# counts.shape = [32, 27] and counts_sum_inv.shape = [32, 1]
# When doing deriv wrt counts_sum_inv, its shape was (32, 1), so each of its
# values was *broadcast across the 27 values* in counts.shape's 2nd dim (32, 27)
# In this case, we are doing deriv wrt counts, *which doesn't broadcast*, since
# its shape is the larger one, therefore gradients do not accumulate, and 
# we do not have to add them.
# The next complication here is that *we are not actually done calculating
# the gradient for counts yet*. This is because the term is used in another 
# node of the computational graph - namely, in:
# counts_sum = counts.sum(1, keepdims=True)
# Therefore, we have to wait until we calculate that one (and accumulate it
# with this one) before we check our gradient calculation.
dcounts = counts_sum_inv * dprobs

# counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
# deriv of exponent, from wiki
dcounts_sum = -1*counts_sum**(-1-1) * dcounts_sum_inv
cmp('counts_sum', dcounts_sum, counts_sum)

# counts_sum = counts.sum(1, keepdims=True)
# Now we need to calculate the 2nd node in which counts is involved.
# We already calculated the first contribution to its gradient above, so 
# we add this calculation to it.
# The partial derivative of an addition operation is just 1, and we apply 
# the chain rule.
dcounts += 1 * dcounts_sum
# could also do (same result d/t broadcasting):
# dcounts += torch.one_like(counts) * dcounts_sum
cmp('counts', dcounts, counts)

# counts = norm_logits.exp()
# In pytorch, torch.exp() is e^x. The derivative of e^x is, interestingly,
# just e^x per wiki.
dnorm_logits = norm_logits.exp() * dcounts
cmp('norm_logits', dnorm_logits, norm_logits)

# norm_logits = logits - logit_maxes # subtract max for numerical stability
# The deriv of a-b with respect to b is just -1 (see proof on paper).
# However, we are doing broadcasting here because the shapes of the 
# operands are different, so we need to be careful to accumulate when needed.
# logits.shape = (32, 27), logit_maxes.shape = (31, 1)
# Since we are doing dlogit_maxes, and it is broadcast, we need to accumulate
# its gradient.
dlogit_maxes = (-1 * dnorm_logits).sum(1, keepdims=True)
cmp('logit_maxes', dlogit_maxes, logit_maxes)

# norm_logits = logits - logit_maxes # subtract max for numerical stability
# Deriv of a-b wrt a is just 1. logits is the larger shape, so it is not
# broadcast, so we don't accumulate gradient.
# logits is used twice, so we can't check it until accumulating
# the partial derivative at its other node.
dlogits = 1 * dnorm_logits

# logit_maxes = logits.max(1, keepdim=True).values
# The derivative here is a bit wonky, but basically we need to match the 
# shape of logits in its derivative, make it all zeroes, then 
# set the position of the max value to 1.
# The derivative wrt the max value is 1, and the deriv wrt all non-max values
# is 0.
# Make sure to accumulate the gradient with that calculated in the prior node.
dlogits += F.one_hot(torch.argmax(logits, dim=1), num_classes=logits.size(1)) * dlogit_maxes
cmp('logits', dlogits, logits)

# logits = h @ W2 + b2 # output layer
# This one is basically tedious calculus. 
# Ultimately what you are doing is writing out what the matrix multiply
# and vector addition represents in terms of tensors.
# Matmul is just the particular order of multiplication and addition
# of 2 tensors, then the addition involves broadcasting addition.
# As it turns out, the derivitave of matmul is matmul, but changing the terms
# and transposing one of them.
# He said he never remembers this formula, and you don't need to  -
# there is acutally only 1 way to get the correct shapes for each of these.
# e.g., dh needs to match h's shape, which is (32, 64)
# dlogits is (32, 27) and W2 is (64, 27), so the only way to arrive
# at the desired shape is to transpose W2 and matmul it with dlogits.
dh = dlogits @ W2.T
cmp('h', dh, h)

# Likewise here.
dW2 = h.T @ dlogits
cmp('W2', dW2, W2)

# You also know what shape db2 needs to be (same as b2), and the only way
# to get it is by adding dlogits across dim 0.
db2 = dlogits.sum(0)
cmp('b2', db2, b2)

# h = torch.tanh(hpreact) # hidden layer
# We know the deriv of tanh(x) is (1 - tanh(x)**2) from wiki.
# The only trick here is to know which gradient to use the chain rule with.
# Basically, you use the associated term - since we're working with h, we use
# dh.
dhpreact = (1 - h**2) * dh
cmp('hpreact', dhpreact, hpreact)

# hpreact = bngain * bnraw + bnbias
# Working this out by hand, we know the deriv wrt bngain is just bnraw.
# Then, since bngain is broadcast, we need to accumulate it along the 
# appropriate dimension.
dbngain = (bnraw * dhpreact).sum(0, keepdims=True)
cmp('bngain', dbngain, bngain)

# Working out by hand, bnbias is addition, so the derivative is just 1.
# The pattern with simple linear functions seems to be that all other terms
# are effectively discarded, and the differntiation is just the result of 
# differentiating the operation on the variable we are differntiating wrt.
dbnbias = (1 * dhpreact).sum(0, keepdims=True)
cmp('bnbias', dbnbias, bnbias)

# Same as dbngain, except bnraw is not broadcast, so we don't need to 
# accumulate the gradient.
dbnraw = bngain * dhpreact
cmp('bnraw', dbnraw, bnraw)

# bnraw = bndiff * bnvar_inv
# Simpler version of the formula we just did
dbnvar_inv = (bndiff * dbnraw).sum(0, keepdims=True)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)

# No need to broadcast.
# Can't check dbndiff yet since bndiff is used in another node.
dbndiff = bnvar_inv * dbnraw

# bnvar_inv = (bnvar + 1e-5)**-0.5
dbnvar = (2*bnvar + 2*1e-5) * dbnvar_inv
cmp('bnvar', dbnvar, bnvar)

# cmp('bndiff2', dbndiff2, bndiff2)
# cmp('bndiff', dbndiff, bndiff)
# cmp('bnmeani', dbnmeani, bnmeani)
# cmp('hprebn', dhprebn, hprebn)
# cmp('embcat', dembcat, embcat)
# cmp('W1', dW1, W1)
# cmp('b1', db1, b1)
# cmp('emb', demb, emb)
# cmp('C', dC, C)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: True  | approximate: True  | maxdiff: 0.0
h               | exact: True  | approximate: True  | maxdiff: 0.0
W2              | exact: True  | approximate: True  | maxdiff: 0.0
b2              | exact: True  | approximate: True  | maxdiff: 0.0
hpreact         | exact: False | approximate: True  | maxdiff: 9.313225746154785e-10
bngain          | exact: False | approximate: True  | maxdiff: 2.7939677238464355e-09
bnbias          | exact: False | approximate: True  | maxdiff: 3.725290298461914e-09
bnraw  

In [149]:
bndiff.shape, bnvar_inv.shape

(torch.Size([32, 64]), torch.Size([1, 64]))

In [129]:
h.shape, W2.shape, (h@W2).shape

(torch.Size([32, 64]), torch.Size([64, 27]), torch.Size([32, 27]))

In [120]:
logits

tensor([[ 7.5402e-01,  1.0867e+00, -5.3148e-01,  3.9063e-01, -5.2998e-01,
          9.1098e-01, -2.8691e-01,  8.5751e-02, -5.2546e-01, -6.2366e-03,
          6.6833e-02,  1.3939e-01,  1.9463e-01, -7.4573e-02,  5.3713e-02,
         -8.5927e-01, -1.2616e+00, -5.4756e-01, -6.2535e-01,  4.7840e-01,
          4.8583e-01, -4.9247e-01, -2.2429e-01,  8.0495e-01,  5.6572e-01,
         -2.6904e-01, -4.0774e-01],
        [ 2.5293e-01,  4.0351e-01,  8.2224e-01,  3.3540e-01, -9.9663e-02,
         -2.0954e-01, -7.4613e-01,  1.1775e-01, -7.4308e-01, -5.0629e-01,
          1.7587e-01,  7.5877e-02,  2.3366e-01, -2.3688e-01, -1.1014e-03,
         -1.7747e-01, -4.3673e-01, -9.1811e-01, -6.3834e-01,  1.2627e-02,
         -6.7823e-01, -6.3396e-01, -9.2972e-01,  4.2295e-01, -5.9126e-01,
         -1.1248e-01, -5.1765e-01],
        [-5.9415e-01, -2.3434e-01, -7.9519e-01, -9.4140e-01, -4.0218e-01,
          2.3699e-01,  5.5787e-01,  5.9457e-01,  5.3490e-01, -9.7205e-02,
         -6.0697e-01,  8.8207e-02,  2.27

In [119]:
F.one_hot(torch.argmax(logits, dim=1), num_classes=logits.size(1))

tensor([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
         0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
         0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0],


In [114]:
logits

tensor([[ 7.5402e-01,  1.0867e+00, -5.3148e-01,  3.9063e-01, -5.2998e-01,
          9.1098e-01, -2.8691e-01,  8.5751e-02, -5.2546e-01, -6.2366e-03,
          6.6833e-02,  1.3939e-01,  1.9463e-01, -7.4573e-02,  5.3713e-02,
         -8.5927e-01, -1.2616e+00, -5.4756e-01, -6.2535e-01,  4.7840e-01,
          4.8583e-01, -4.9247e-01, -2.2429e-01,  8.0495e-01,  5.6572e-01,
         -2.6904e-01, -4.0774e-01],
        [ 2.5293e-01,  4.0351e-01,  8.2224e-01,  3.3540e-01, -9.9663e-02,
         -2.0954e-01, -7.4613e-01,  1.1775e-01, -7.4308e-01, -5.0629e-01,
          1.7587e-01,  7.5877e-02,  2.3366e-01, -2.3688e-01, -1.1014e-03,
         -1.7747e-01, -4.3673e-01, -9.1811e-01, -6.3834e-01,  1.2627e-02,
         -6.7823e-01, -6.3396e-01, -9.2972e-01,  4.2295e-01, -5.9126e-01,
         -1.1248e-01, -5.1765e-01],
        [-5.9415e-01, -2.3434e-01, -7.9519e-01, -9.4140e-01, -4.0218e-01,
          2.3699e-01,  5.5787e-01,  5.9457e-01,  5.3490e-01, -9.7205e-02,
         -6.0697e-01,  8.8207e-02,  2.27

In [ ]:
# Exercise 2: backprop through cross_entropy but all in one go
# to complete this challenge look at the mathematical expression of the loss,
# take the derivative, simplify the expression, and just write it out

# forward pass

# before:
# logit_maxes = logits.max(1, keepdim=True).values
# norm_logits = logits - logit_maxes # subtract max for numerical stability
# counts = norm_logits.exp()
# counts_sum = counts.sum(1, keepdims=True)
# counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
# probs = counts * counts_sum_inv
# logprobs = probs.log()
# loss = -logprobs[range(n), Yb].mean()

# now:
loss_fast = F.cross_entropy(logits, Yb)
print(loss_fast.item(), 'diff:', (loss_fast - loss).item())

In [ ]:
# backward pass

# -----------------
# YOUR CODE HERE :)
dlogits = None # TODO. my solution is 3 lines
# -----------------

#cmp('logits', dlogits, logits) # I can only get approximate to be true, my maxdiff is 6e-9

In [ ]:
# Exercise 3: backprop through batchnorm but all in one go
# to complete this challenge look at the mathematical expression of the output of batchnorm,
# take the derivative w.r.t. its input, simplify the expression, and just write it out
# BatchNorm paper: https://arxiv.org/abs/1502.03167

# forward pass

# before:
# bnmeani = 1/n*hprebn.sum(0, keepdim=True)
# bndiff = hprebn - bnmeani
# bndiff2 = bndiff**2
# bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
# bnvar_inv = (bnvar + 1e-5)**-0.5
# bnraw = bndiff * bnvar_inv
# hpreact = bngain * bnraw + bnbias

# now:
hpreact_fast = bngain * (hprebn - hprebn.mean(0, keepdim=True)) / torch.sqrt(hprebn.var(0, keepdim=True, unbiased=True) + 1e-5) + bnbias
print('max diff:', (hpreact_fast - hpreact).abs().max())

In [ ]:
# backward pass

# before we had:
# dbnraw = bngain * dhpreact
# dbndiff = bnvar_inv * dbnraw
# dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
# dbnvar = (-0.5*(bnvar + 1e-5)**-1.5) * dbnvar_inv
# dbndiff2 = (1.0/(n-1))*torch.ones_like(bndiff2) * dbnvar
# dbndiff += (2*bndiff) * dbndiff2
# dhprebn = dbndiff.clone()
# dbnmeani = (-dbndiff).sum(0)
# dhprebn += 1.0/n * (torch.ones_like(hprebn) * dbnmeani)

# calculate dhprebn given dhpreact (i.e. backprop through the batchnorm)
# (you'll also need to use some of the variables from the forward pass up above)

# -----------------
# YOUR CODE HERE :)
dhprebn = None # TODO. my solution is 1 (long) line
# -----------------

cmp('hprebn', dhprebn, hprebn) # I can only get approximate to be true, my maxdiff is 9e-10

In [ ]:
# Exercise 4: putting it all together!
# Train the MLP neural net with your own backward pass

# init
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 200 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

# same optimization as last time
max_steps = 200000
batch_size = 32
n = batch_size # convenience
lossi = []

# use this context manager for efficiency once your backward pass is written (TODO)
#with torch.no_grad():

# kick off optimization
for i in range(max_steps):

  # minibatch construct
  ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
  Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

  # forward pass
  emb = C[Xb] # embed the characters into vectors
  embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
  # Linear layer
  hprebn = embcat @ W1 + b1 # hidden layer pre-activation
  # BatchNorm layer
  # -------------------------------------------------------------
  bnmean = hprebn.mean(0, keepdim=True)
  bnvar = hprebn.var(0, keepdim=True, unbiased=True)
  bnvar_inv = (bnvar + 1e-5)**-0.5
  bnraw = (hprebn - bnmean) * bnvar_inv
  hpreact = bngain * bnraw + bnbias
  # -------------------------------------------------------------
  # Non-linearity
  h = torch.tanh(hpreact) # hidden layer
  logits = h @ W2 + b2 # output layer
  loss = F.cross_entropy(logits, Yb) # loss function

  # backward pass
  for p in parameters:
    p.grad = None
  loss.backward() # use this for correctness comparisons, delete it later!

  # manual backprop! #swole_doge_meme
  # -----------------
  # YOUR CODE HERE :)
  dC, dW1, db1, dW2, db2, dbngain, dbnbias = None, None, None, None, None, None, None
  grads = [dC, dW1, db1, dW2, db2, dbngain, dbnbias]
  # -----------------

  # update
  lr = 0.1 if i < 100000 else 0.01 # step learning rate decay
  for p, grad in zip(parameters, grads):
    p.data += -lr * p.grad # old way of cheems doge (using PyTorch grad from .backward())
    #p.data += -lr * grad # new way of swole doge TODO: enable

  # track stats
  if i % 10000 == 0: # print every once in a while
    print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
  lossi.append(loss.log10().item())

  if i >= 100: # TODO: delete early breaking when you're ready to train the full net
    break

In [ ]:
# useful for checking your gradients
# for p,g in zip(parameters, grads):
#   cmp(str(tuple(p.shape)), g, p)

In [ ]:
# calibrate the batch norm at the end of training

with torch.no_grad():
  # pass the training set through
  emb = C[Xtr]
  embcat = emb.view(emb.shape[0], -1)
  hpreact = embcat @ W1 + b1
  # measure the mean/std over the entire training set
  bnmean = hpreact.mean(0, keepdim=True)
  bnvar = hpreact.var(0, keepdim=True, unbiased=True)


In [ ]:
# evaluate train and val loss

@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte),
  }[split]
  emb = C[x] # (N, block_size, n_embd)
  embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
  hpreact = embcat @ W1 + b1
  hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
  h = torch.tanh(hpreact) # (N, n_hidden)
  logits = h @ W2 + b2 # (N, vocab_size)
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())

split_loss('train')
split_loss('val')

In [ ]:
# I achieved:
# train 2.0718822479248047
# val 2.1162495613098145

In [ ]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
    
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      # forward pass
      emb = C[torch.tensor([context])] # (1,block_size,d)      
      embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
      hpreact = embcat @ W1 + b1
      hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
      h = torch.tanh(hpreact) # (N, n_hidden)
      logits = h @ W2 + b2 # (N, vocab_size)
      # sample
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break
    
    print(''.join(itos[i] for i in out))